# Doc de la lib construite pour explorer PORTIC
 
Objectif de cette lib : abstraire le fonctionnement des APIs de PORTIC et de TOFLIT18 à travers un client facile à manipuler.

    Voir la doc de l'API PORTIC ici : https://gitlab.huma-num.fr/portic/porticapi/-/tree/master/porticapi 
    Voir le code de l'API toflit18 ici : https://github.com/medialab/toflit18
    
 Résumé des fonctions proposées :
 
#### Fonctions principales (résultats directement utilisables pour l'étude) :
Portic:
- get_pointcalls : récupération de pointcalls (infos disponibles : année, port d'enregistrement, entrée/sortie dans le port, type de navire, activité déclarée (pêche, transport de marchandises ...))
- get_flows : récupération de flux (1 flux portic = 1 trajet intermédiaire (escale), souvent décrit par plusieurs pointcalls)
- get_travels : récupération de travels (1 travel = ensemble de flux/escales qui s'enchainent ; infos disponibles : point de départ, d'arrivée, distance parcourue, infos sur le navire ...)

Toflit :
- get_flows : récupération de flows (1 flow = import / export d'une marchandise ; infos disponibles : import ou export, produit commercé, partenaire commercial, bureau des Fermes ou a été enregistré le flux ...)
- get_flows_per_year : récupère les flux par année par direction ou par type de source
- get_locations : récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges 
- get_time_series : récupère des séries temporelles à propos des flux de marchandises



 
#### Fonctions secondaires (résultats qui vont plutôt aider à paramétrer les fonctions principales, nourrir des fonctions de visualisation, ...)
Portic:
- get_fieldnames : récupère les noms des variables des données et leur description
- get_departures_details :
- get_destinations_aggregated : 
- get_ports : description de ports et de leur localisation géographique

Toflit :
- get_customs_regions() : récupère toutes les direction des Fermes de la base Toflit18
- get_product_classifications : récupère des informations sur une classification de produit toflit (et optionellement sur ses classifications filles)
- get_partner_classifications : récupère des informations sur une classification de partenaires toflit (et optionellement sur ses classifications filles)
- get_classification_groups : récupère l'ensemble des catégories associées à une certaine classification (donnée en paramètre)
- get_product_terms : récupère des produits et un nombre de flows associés ???
- get_sources_types : récupère les types de sources disponibles

# Imports et bootstrapping des clients

In [2]:
import networkx as nx
import requests
import csv
import json
from poitousprint import Portic, Toflit, nest_toflit18_flow, nest_portic_pointcall, build_cooccurence_graph

portic_client = Portic()
toflit_client = Toflit()

# Méthodes pour l'accès à PORTIC
pouvant être appelées depuis `portic_client` :


## portic_client.get_fieldnames()


Synopsis:
Récupère les noms des variables des données et leur description (très pratique quand on ne sait plus ce qu'une colonne signifie)
##### regarder attributs name et description

---

### Exemple : je veux tous les noms de colonnes présents dans les données navigo

In [2]:
portic_client.get_fieldnames()

[{'api': 'None',
  'name': 'travel_id',
  'shortname': 't001',
  'type': 'text',
  'description': 'Identifiant du trajet'},
 {'api': 'None',
  'name': 'distance_dep_dest',
  'shortname': 't002',
  'type': 'numeric',
  'description': "Distance en km entre départ et destination, arrondie à 3 decimales, à vol d'oiseau"},
 {'api': 'None',
  'name': 'distance_homeport_dep',
  'shortname': 't003',
  'type': 'numeric',
  'description': "Distance en km entre homeport et départ, arrondie à 3 decimales, à vol d'oiseau"},
 {'api': 'None',
  'name': 'departure',
  'shortname': 't004',
  'type': 'text',
  'description': 'Toponyme du port de départ'},
 {'api': 'None',
  'name': 'departure_fr',
  'shortname': 't005',
  'type': 'text',
  'description': 'Standardised name of the port de départ, in French'},
 {'api': 'None',
  'name': 'departure_en',
  'shortname': 't006',
  'type': 'text',
  'description': 'Standardised name of the port  de départ, in English'},
 {'api': 'None',
  'name': 'departure_uh

## portic_client.get_pointcalls(params)


Synopsis:

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| start_year | int (ou string) | | | année minimale d'enregistrement du pointcall |
| end_year | int (ou string) | | | année maximale d'enregistrement du pointcall |
| year | int (ou string) | | | année exacte d'enregistrement du pointcall (l'emporte sur params start/end_year |

### Exemple : je veux tous les pointcalls enregistrés dans le Poitou en 1789 

In [6]:
portic_client.get_pointcalls(
 year = 1789,
 source_subset = 'Poitou_1789'
)[0:10]

[{'pkid': 94374,
  'record_id': '00350929',
  'pointcall': 'Marennes',
  'pointcall_uhgs_id': 'A0136930',
  'toponyme_fr': 'Marennes',
  'toponyme_en': 'Marennes',
  'latitude': '45.816667',
  'longitude': '-1.116667',
  'pointcall_admiralty': 'Marennes',
  'pointcall_province': 'Saintonge',
  'pointcall_states': '[{"1749-1815" : "France"}]',
  'pointcall_substates': None,
  'pointcall_states_en': '[{"1749-1815" : "France"}]',
  'pointcall_substates_en': None,
  'state_1789_fr': 'France',
  'state_1789_en': 'France',
  'substate_1789_fr': None,
  'substate_1789_en': None,
  'source_1787_available': True,
  'source_1789_available': True,
  'pointcall_status': 'siège amirauté',
  'shiparea': 'ACE-ROCH',
  'pointcall_point': '0101000020110F0000CC2247D42C59FEC031FEA89F3AF05541',
  'ferme_direction': 'La Rochelle',
  'ferme_bureau': 'Marennes',
  'ferme_bureau_uncertainty': 0.0,
  'partner_balance_1789': None,
  'partner_balance_supp_1789': 'France',
  'partner_balance_1789_uncertainty': No

## portic_client.get_flows(params)

Synopsis:

Retourne une liste de flux, c'est-à-dire de voyages liés à des ports spécifiques, soit en y entrant (direction "in"), soit en en sortant (direction "out"), soit en ayant navigué autour (direction "in-out")

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| ports | string | UHGS_id | 4326 | liste des ids de ports à filtrer (séparés par des virgules)


### Exemple 

In [7]:
# Exemple de requête : tous les bateaux passant par Bordeaux (A0180923) et Boulogne sur Mer (A0152606)

resp = portic_client.get_flows(
 # ports = 'A1964694'
 # ports = ['A1964694', 'A0171758','A0136930', 'A0196496', 'A0198999']
 # 'params': [ 'travel_rank', 'ship_id', 'departure', 'destination', 'travel_uncertainty', 'distance_dep_dest'],
 departure = 'La Rochelle'
)
resp
# print(json.dumps(resp[0:10], indent=2))

TypeError: 'NoneType' object is not iterable

## portic_client.get_travels(params)


Synopsis:

Récupère les données de trajectoires calculées.

#### possibilité de filtrer à une valeur ou une liste de valeurs par attribut
---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|


### Exemple : tous les voyages au départ d'un port rattaché à la direction des Fermes de La Rochelle, dont le port d'attache du navire est Dieppe et Noirmoutier, et son pavillon français, ...

In [26]:
portic_client.get_travels(
    departure_action='Out',
    departure_ferme_direction='La Rochelle',
    homeport_toponyme_fr=['Dieppe', 'Noirmoutier'],
    ship_flag_standardized_en='French'
)[0:10]

[{'travel_id': '0003811N- 21',
  'distance_dep_dest': 85.603,
  'distance_homeport_dep': 0.0,
  'departure': 'Noirmoutier',
  'departure_fr': 'Noirmoutier',
  'departure_en': 'Noirmoutier',
  'departure_uhgs_id': 'A0136403',
  'departure_latitude': '47',
  'departure_longitude': '-2.25',
  'departure_admiralty': 'Sables-d’Olonne',
  'departure_province': 'Poitou',
  'departure_states': '[{"1749-1815" : "France"}]',
  'departure_substates': None,
  'departure_state_1789_fr': 'France',
  'departure_substate_1789_fr': None,
  'departure_state_1789_en': 'France',
  'departure_substate_1789_en': None,
  'departure_ferme_direction': 'La Rochelle',
  'departure_ferme_bureau': "Sables d'Olonne",
  'departure_ferme_bureau_uncertainty': 0.0,
  'departure_partner_balance_1789': 'Petites Iles',
  'departure_partner_balance_supp_1789': 'France',
  'departure_partner_balance_1789_uncertainty': -1.0,
  'departure_partner_balance_supp_1789_uncertainty': 0.0,
  'departure_shiparea': 'ACE-YEUX',
  'depa

## portic_client.get_departures_details(params)


Synopsis:

Retourne le détail des *voyages* au départ des points situés dans le voisinage (voir paramètre radius) du point requêté.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple : description 

In [10]:
portic_client.get_departures_details(
 lat = 46.151913,
 lon = -1.172415,
 radius = 30
)[0:10]

[{'departure': 'La Rochelle',
  'departure_uhgs_id': 'A0198999',
  'departure_latitude': '46.166667',
  'departure_longitude': '-1.15'},
 {'departure': 'La Prée',
  'departure_uhgs_id': 'A0179464',
  'departure_latitude': '46.016667',
  'departure_longitude': '-1.35'},
 {'departure': 'dans le Pertuis',
  'departure_uhgs_id': 'A1968950',
  'departure_latitude': '46.044980',
  'departure_longitude': ' -1.204802'},
 {'departure': 'Le Plomb',
  'departure_uhgs_id': 'A0170183',
  'departure_latitude': '46.2',
  'departure_longitude': '-1.2'},
 {'departure': 'Esnandes',
  'departure_uhgs_id': 'A0213721',
  'departure_latitude': '46.25',
  'departure_longitude': '-1.083333'},
 {'departure': "L' Houmée",
  'departure_uhgs_id': 'A0152629',
  'departure_latitude': '46.193652',
  'departure_longitude': '-1.186724'},
 {'departure': 'Rade de La Pallice',
  'departure_uhgs_id': 'A0154161',
  'departure_latitude': '46.165762',
  'departure_longitude': '-1.201386'},
 {'departure': 'Ile de Ré',
  'depa

## portic_client.get_destinations_aggregated(params)

Synopsis:

Retourne une aggrégation des *voyages* à l'arrivée des points situés dans le voisinage (voir paramètre radius) du point requêté.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple

In [11]:
portic_client.get_destinations_aggregated(
 lat = 45.2333,
 lon = -1,
 radius = 50
)[0:60]

[{'label': 'Bayonne', 'value': 1, 'id': 'Bayonne'},
 {'label': 'Bordeaux', 'value': 26, 'id': 'Bordeaux'},
 {'label': 'Brest', 'value': 43, 'id': 'Brest'},
 {'label': 'Cherbourg', 'value': 10, 'id': 'Cherbourg'},
 {'label': 'Dunkerque', 'value': 1, 'id': 'Dunkerque'},
 {'label': 'Granville', 'value': 1, 'id': 'Granville'},
 {'label': 'Isigny', 'value': 1, 'id': 'Isigny'},
 {'label': 'La Hougue', 'value': 1, 'id': 'La Hougue'},
 {'label': 'La Rochelle', 'value': 4, 'id': 'La Rochelle'},
 {'label': 'Marennes', 'value': 4, 'id': 'Marennes'},
 {'label': 'Morlaix', 'value': 29, 'id': 'Morlaix'},
 {'label': 'Quimper', 'value': 8, 'id': 'Quimper'},
 {'label': 'Rouen', 'value': 2, 'id': 'Rouen'},
 {'label': 'Saint-Brieuc', 'value': 1, 'id': 'Saint-Brieuc'},
 {'label': 'Saint-Malo', 'value': 12, 'id': 'Saint-Malo'},
 {'label': 'Vannes', 'value': 11, 'id': 'Vannes'},
 {'label': None, 'value': 1, 'id': None}]

## portic_client.get_ports(params)

 Synopsis:
 Retourne une liste de *ports_points* au format JSON

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
|srid|int|---|---|identifiant de référence spatiale unique associé à un système de coordonnées, une tolérance et une résolution spécifiques|


### Exemple

In [12]:
resp = portic_client.get_ports(
    srid = 4326
)
print(json.dumps(resp[0:1], indent=2))

[
  {
    "ogc_fid": 11,
    "uhgs_id": "A1965015",
    "total": 13,
    "toponym": "Bourg [de l' Ile aux Moines]",
    "belonging_states": "[{\"1749-1815\" : \"France\"}]",
    "belonging_substates": null,
    "status": null,
    "geonameid": 6620139.0,
    "admiralty": "Vannes",
    "province": "Bretagne",
    "shiparea": "ACE-IROI",
    "point": "{\"type\":\"Point\",\"coordinates\":[-2.843156,47.598946]}"
  }
]


# Méthodes pour l'accès à TOFLIT
pouvant être appelées depuis `toflit_client` :


In [6]:
result = toflit_client.get_flows(
    year=1792, 
    start_year=1789, 
    end_year=1789, 
    product_orthographic='dentelle de soie'
)
flow = result[0]
# print(flow)

nest_toflit18_flow(flow)

{'flow': {'year': '1792', 'export_import': 'Exports'},
 'source': {'source_type': '1792-both semester',
  'best_guess_national_prodxpart': '0',
  'best_guess_national_partner': '0',
  'best_guess_national_product': '0',
  'best_guess_region_prodxpart': '0',
  'best_guess_national_region': '0',
  'nbr_obs': '497',
  'imprimatur': 'x',
  'filepath': 'sources/1792-both semester/AN_F12_252_et_F12_1843B_Exports_1792.csv',
  'source': 'AN F12 252 et F12 1843B',
  'sheet': '26'},
 'product_origin': {'origin': '',
  'origin_origin': '',
  'customs_region_origin': '',
  'details_provenance': ''},
 'product_quantity_and_value': {'value': '',
  'quantity': '7069',
  'computed_value': '0',
  'value_as_reported': '',
  'computed_value_per_unit': '0',
  'computed_quantity': '0',
  'value_minus_unit_val_x_qty': '',
  'quantities_metric': '3460.2754',
  'unit_price_metric': '',
  'conv_simplification_to_metric': '.48949999',
  'quantity_unit_metric': 'kg',
  'quantity_unit_simplification': 'livres',
 

## toflit_client.get_customs_regions()

Synopsis :

Récupère toutes les customs region (direction des Fermes) de la base

---

### Exemple

In [15]:
toflit_client.get_customs_regions()

[{'id': 'Amiens', 'name': 'Amiens'},
 {'id': 'Auch', 'name': 'Auch'},
 {'id': 'Bayonne', 'name': 'Bayonne'},
 {'id': 'Besançon', 'name': 'Besançon'},
 {'id': 'Bordeaux', 'name': 'Bordeaux'},
 {'id': 'Bourgogne', 'name': 'Bourgogne'},
 {'id': 'Caen', 'name': 'Caen'},
 {'id': 'Charleville', 'name': 'Charleville'},
 {'id': 'Châlons', 'name': 'Châlons'},
 {'id': "Colonies_Françaises_de_l'Amérique",
  'name': "Colonies Françaises de l'Amérique"},
 {'id': 'Dunkerque', 'name': 'Dunkerque'},
 {'id': 'Flandre', 'name': 'Flandre'},
 {'id': 'France', 'name': 'France'},
 {'id': 'France_par_la_Compagnie_des_Indes',
  'name': 'France par la Compagnie des Indes'},
 {'id': 'Grenoble', 'name': 'Grenoble'},
 {'id': 'La_Rochelle', 'name': 'La Rochelle'},
 {'id': 'Langres', 'name': 'Langres'},
 {'id': 'Le_Havre', 'name': 'Le Havre'},
 {'id': 'Lorient', 'name': 'Lorient'},
 {'id': 'Lyon', 'name': 'Lyon'},
 {'id': 'Marseille', 'name': 'Marseille'},
 {'id': 'Montpellier', 'name': 'Montpellier'},
 {'id': 'Nan

## toflit_client.get_product_classifications(classification)

Synopsis :

Récupère les classifications de produits

---

### Exemple

In [17]:
resp = toflit_client.get_product_classifications()
# removing children for the sake of logging
resp['children'] = None
print(json.dumps(resp, indent=2))

{
  "name": "Source",
  "description": "the product names as transcribed from archive volumes",
  "model": "product",
  "source": true,
  "id": "product_source",
  "slug": "source",
  "author": "TOFLIT 18",
  "parent": null,
  "groupsCount": 60193,
  "itemsCount": null,
  "unclassifiedItemsCount": 1,
  "completion": 0,
  "children": null
}


## toflit_client.get_partner_classifications()

Synopsis :

Récupère les classifications de partenaires

---

### Exemple

In [18]:
resp = toflit_client.get_partner_classifications()
# removing children for the sake of logging
resp['children'] = None
print(json.dumps(resp, indent=2))

{
  "name": "Source",
  "description": "the partner names as transcribed from archive volumes",
  "model": "partner",
  "source": true,
  "id": "partner_source",
  "slug": "source",
  "author": "TOFLIT 18",
  "parent": null,
  "groupsCount": 1042,
  "itemsCount": null,
  "unclassifiedItemsCount": 1,
  "completion": 0,
  "children": null
}


## toflit_client.get_classification_groups(classification)

Synopsis :

Récupère l'ensemble des catégories associées à une classification en particulier (sans le détail des valeurs)

---

### Exemple

In [19]:
toflit_client.get_classification_groups('partner_grouping')[0:5]

[{'id': '????~partner_grouping', 'name': '????'},
 {'id': 'Afrique~partner_grouping', 'name': 'Afrique'},
 {'id': 'Allemagne~partner_grouping', 'name': 'Allemagne'},
 {'id': 'Amériques~partner_grouping', 'name': 'Amériques'},
 {'id': 'Angleterre~partner_grouping', 'name': 'Angleterre'}]

## toflit_client.get_classification_search(classification)

Synopsis :

Récupère le détail des groupements associés à une classification en particulier.

---

Paramètre d'URL `classification` : le nom de la classification préfixé par son type (ex. "product_simplification", ou "partner_source")

---

### Exemple

In [ ]:
toflit_client.get_classification_search('product_orthographic')[0:2]

## toflit_client.get_locations(classification, params)

Synopsis :

Récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges

---

Paramètre `classification` : l'id de la classification de partenaire à utiliser

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |

---

### Exemple

In [ ]:
client.toflit.get_locations(
    classification = 'partner_grouping', 
    dateMin = 1750,
    dateMax = 1760,
    kind = 'total',
    sourceType = 'Best Guess national product x partner'
)

## toflit_client.get_flows_per_year(type, params)

Synopsis :

Récupère les flux par année par direction ou par type de source

---

Paramètre `type` : le type de flux 'direction' ou 'sourceType'

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
toflit_client.get_flows_per_year(
    type ='sourceType', 
    dateMin = 1750,
    dateMax = 1780,
    sourceType = 'National toutes directions partenaires manquants'
)

## toflit_client.get_time_series(params)

Synopsis :

Récupère des séries temporelles à propos des flux de marchandises

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
toflit_client.get_time_series(
    direction = 'La_Rochelle'
)[0:2]

## toflit_client.get_product_terms(classification, params)

Synopsis :

Récupère des séries temporelles à propos des flux de marchandises

---

Paramètre `classification` : l'id de la classification de produit à utiliser

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| child | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| childClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
toflit_client.get_product_terms(
 'product_simplification', 
  # sourceType = "National best guess",
 childClassification = product_simplification,
 dateMin = 1750,
 dateMax = 1760,
)['data'][0:3]

## toflit_client.get_sources_types()

Synopsis :

Récupère les types de sources disponibles

---

### Exemple

In [16]:
toflit_client.get_sources_types()

['1792-both semester',
 '1792-first semester',
 'Best Guess customs region product x partner',
 'Best Guess national customs region',
 'Best Guess national partner',
 'Best Guess national product',
 'Best Guess national product x partner',
 'Compagnie des Indes',
 'Local',
 'National partenaires manquants',
 'National toutes directions partenaires manquants',
 'National toutes directions sans produits',
 'National toutes directions tous partenaires',
 'Objet Général',
 'Résumé',
 'Tableau Général',
 'Tableau des quantités']